## BIOSTAT 257 Homework 2

Consider a linear mixed effects model

$$Y_i = X_i\beta + Z_i\gamma + \epsilon_i, \quad i = 1,\ldots,n$$

where

- $Y_i \in \mathbb{R}^{n_i}$ is the reponse vector of the $i$-th individual,
- $X_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of  $i$-th individual,
- $Z_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of  $i$-th individual, 
- $\epsilon_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(0_{n_i}, \sigma^2 I_{n_i})$,
- $\beta \in \mathbb{R}^{p}$ are fixed effects, and
- $\gamma \in \mathbb{R}^{q}$ are random effects assumed to be $N(0_{q}, \Sigma_{q \times q})$ independent of $\epsilon_i$.

### Question 1: Formula

Write down the log-likelihood of the  $i$-th datum  $(Y_i,X_i,Z_i)$  given parameters $(\beta,\Sigma,\sigma^2)$.

## Question 2: 